<a href="https://www.kaggle.com/code/kriti83/market-prediction?scriptVersionId=279974718" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hull-tactical-market-prediction/train.csv
/kaggle/input/hull-tactical-market-prediction/test.csv
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_inference_server.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/default_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/hull-tactical-market-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py
/kaggl

In [2]:
import os
import pandas as pd
import numpy as np
import polars as pl  # ← ADDED
from sklearn.ensemble import RandomForestRegressor
import kaggle_evaluation.default_inference_server

df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
0,0,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002421,0.000301,-0.003038
1,1,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008495,0.000303,-0.009114
2,2,0,0,0,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009624,0.000301,-0.010243
3,3,0,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004662,0.000299,0.004046
4,4,0,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011686,0.000299,-0.012301


In [3]:
# 1. Handle missing values
missing_pct = df.isnull().sum() / len(df)
df = df.loc[:, missing_pct < 0.3] 
print(df.shape)

(9021, 86)


# Feature Engineering

In [4]:
def feature_engineering(data):
    """Apply feature engineering - works with Pandas"""
    data = data.copy()

    # Lag features
    data['return_lag1'] = data['forward_returns'].shift(1)
    data['return_lag2'] = data['forward_returns'].shift(2)
    data['return_lag5'] = data['forward_returns'].shift(5)

    # Rolling
    data['return_mean_20'] = data['forward_returns'].rolling(20).mean()
    data['return_std_20'] = data['forward_returns'].rolling(20).std()

    # Momentum
    data['momentum_5'] = data['forward_returns'].rolling(5).sum()
    data['momentum_20'] = data['forward_returns'].rolling(20).sum()

    # Volatility ratio
    data['vol_ratio'] = data['return_std_20'] / data['return_std_20'].rolling(60).mean()

    # Market regime
    data['positive_days_20'] = (data['forward_returns'] > 0).rolling(20).sum()

    return data

# Apply feature engineering
df = feature_engineering(df)

# Remove NaN created by rolling windows
df = df.dropna()
print(f"After dropna: {len(df)} rows")

After dropna: 7003 rows


# Create Target

In [5]:
def create_allocation_target(returns, market_returns):
    allocation = np.ones(len(returns))
    allocation[returns > 0.01] = 1.5
    allocation[returns > 0.02] = 2.0
    allocation[returns < -0.005] = 0.5
    allocation[returns < -0.01] = 0.0
    return allocation

df['target_allocation'] = create_allocation_target(
    df['forward_returns'], 
    df['market_forward_excess_returns']
)

# Split data

In [6]:
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

train = df.iloc[:train_size]
val = df.iloc[train_size:train_size+val_size]

feature_cols = [col for col in df.columns if col not in 
                ['date_id', 'forward_returns', 'risk_free_rate', 
                 'market_forward_excess_returns', 'target_allocation']]

X_train = train[feature_cols]
y_train = train['target_allocation']
X_val = val[feature_cols]
y_val = val['target_allocation']

print(f"Training features: {len(feature_cols)}")
print(f"Train size: {len(X_train)}")
print(f"Val size: {len(X_val)}")

Training features: 91
Train size: 4902
Val size: 1050


# Train Validation Model

In [7]:
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
allocations_rf = rf_model.predict(X_val)
print("Validation model trained")

Validation model trained


# Calculate validation score

In [8]:
def modified_sharpe_ratio(strategy_returns, market_returns):
    excess_return = strategy_returns.mean() - market_returns.mean()
    strategy_vol = strategy_returns.std()
    market_vol = market_returns.std()
    sharpe = excess_return / strategy_vol if strategy_vol > 0 else 0
    vol_ratio = strategy_vol / market_vol if market_vol > 0 else 1
    if vol_ratio > 1.2:
        penalty = (vol_ratio - 1.2) * 0.5
        sharpe = sharpe * (1 - penalty)
    return sharpe

strategy_returns = allocations_rf * val['market_forward_excess_returns'].values
score = modified_sharpe_ratio(strategy_returns, val['market_forward_excess_returns'].values)
print(f"Validation Score: {score:.4f}")

Validation Score: 0.1617


# Train Model

In [9]:
print("\nTraining final model on all data...")
X_final = df[feature_cols]
y_final = df['target_allocation']

rf_model_final = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_model_final.fit(X_final, y_final)
print("Final model trained!")


Training final model on all data...
Final model trained!


# Prediction

In [10]:
def predict(test: pl.DataFrame) -> float:
    try:
        test_pd = test.to_pandas()
        
        # Extract features
        features = []
        for col in feature_cols:
            features.append(test_pd[col].iloc[0] if col in test_pd.columns else 0)
        
        # Predict
        allocation = rf_model_final.predict(np.array(features).reshape(1, -1))[0]
        
        # Improvements
        allocation = np.clip(allocation, 0.6, 1.4)
        allocation = 0.85 * allocation + 0.15 * 1.0
        
        return float(np.clip(allocation, 0, 2))
    except:
        return 1.0

In [11]:
print("\nSetting up inference server...")

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in competition mode...")
    inference_server.serve()
else:
    print("Running in local test mode...")
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))


Setting up inference server...
Running in local test mode...


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does